# Data Engineer Technical Assigment
### Author: Konrad Wronski
Description: 
Technical assignment description
You are a data engineer working for a retail company. The company’s database handles large
volumes of data. You are tasked with creating a datalake in ADLS for reporting purposes.
For simplicity, instead of tables, assume files are in a source folder “Sales_Data” in csv format and
instead of ADLS, target location is “cleansed” folder (Create it yourself).
Source folder contains monthly sales data files.
1. Design the process to read all the files from source folder using PySpark, combine them
as a single file, and write it to the cleansed folder.
2. Which file format would you choose to write in the cleansed folder and why?
3. Mention data partitioning strategy you would propose for this table and justify your choice
of partitioning method.
4. Additionally, outline the steps you would take to implement this partitioning strategy,
considering both the technical aspects and potential challenges.
Note: Process all the files in a single run. Ensure that there is no data duplication.
[Hint: Make use of Window function for deduplication]
### Expected results of assignment:
- A notebook or python script.
- A separate documentation file with a brief explanation of the approach, data exploration,
assumptions/considerations, and instructions on how to run the application (if any).
- Output dataset in cleansed folder in your preferred file format.
- Data quality checks (like input/output dataset validation)
### Metyis development guidelines
- We appreciate a combination of Software and Data Engineering good practices.
- Proper logging and exception handling
### Evaluation criteria for results of technical assignment
We use following criteria to evaluate results:
- Well-structured code: we expect maintainability, readability.
- Scalability: Should be able to handle high volumes of data.
- Documentation

## 1. Data Ingestion 

### 1.1 Importing packages

In [2]:
from pyspark.sql import SparkSession
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt


/var/folders/1m/mckmrfpx3yn1g1wj1m8cxm000000gn/T/ipykernel_92932/2147531606.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
